# K-League 패스 좌표 예측 - Colab Setup

> **작성일:** 2025-12-15
> **목적:** Google Colab + GPU 환경 구축
> **전략:** CatBoost 재현 → Gap 확인 → Neural Network 시도

## 1. Google Drive 마운트

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create project folder
project_path = '/content/drive/MyDrive/kleague-algorithm'
os.makedirs(project_path, exist_ok=True)
os.chdir(project_path)

print(f"✅ Working directory: {os.getcwd()}")

## 2. 데이터 업로드

**방법 1: 직접 업로드**
- `train.csv`, `test.csv`, `sample_submission.csv`
- Google Drive의 `kleague-algorithm/` 폴더에 업로드

**방법 2: 로컬에서 복사**

In [ ]:
# Check data files
import glob

csv_files = glob.glob('*.csv')
print(f"CSV files: {csv_files}")

if 'train.csv' not in csv_files:
    print("⚠️ train.csv not found!")
    print("Please upload train.csv to Google Drive")
else:
    print("✅ Data files ready!")

## 3. GPU 확인

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU 사용 가능!")
else:
    print("⚠️ CPU 모드")
    print("런타임 → 런타임 유형 변경 → GPU 선택")

## 4. 라이브러리 설치

In [ ]:
# CatBoost (GPU 지원)
!pip install -q catboost

# TabNet (Tabular Neural Network)
!pip install -q pytorch-tabnet

# FT-Transformer는 직접 구현 필요

print("✅ 라이브러리 설치 완료!")

## 5. FastExperiment 클래스 (로컬 코드 복사)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import time
import warnings
warnings.filterwarnings('ignore')

class FastExperiment:
    """빠른 실험 시스템 (Colab 버전)"""
    
    def __init__(self, sample_frac=1.0, n_folds=5, random_state=42):
        self.sample_frac = sample_frac
        self.n_folds = n_folds
        self.random_state = random_state
        np.random.seed(random_state)
    
    def load_data(self, train_path='train.csv', sample=False):
        """데이터 로드"""
        print(f"\n{'='*80}")
        print("데이터 로드")
        print(f"{'='*80}")
        
        train_df = pd.read_csv(train_path)
        print(f"  원본: {len(train_df):,}개 패스")
        
        if sample:
            episodes = train_df['game_episode'].unique()
            n_sample = int(len(episodes) * self.sample_frac)
            sampled_episodes = np.random.choice(episodes, size=n_sample, replace=False)
            train_df = train_df[train_df['game_episode'].isin(sampled_episodes)]
            print(f"  샘플링: {len(sampled_episodes):,}개 Episode ({self.sample_frac*100:.0f}%)")
        
        return train_df
    
    def create_features(self, df):
        """피처 생성 (Episode 독립성 유지)"""
        df = df.copy()
        
        # Zone 6x6
        df['zone_x'] = (df['start_x'] / (105/6)).astype(int).clip(0, 5)
        df['zone_y'] = (df['start_y'] / (68/6)).astype(int).clip(0, 5)
        
        # Direction
        df['dx'] = df['end_x'] - df['start_x']
        df['dy'] = df['end_y'] - df['start_y']
        df['prev_dx'] = df.groupby('game_episode')['dx'].shift(1).fillna(0)
        df['prev_dy'] = df.groupby('game_episode')['dy'].shift(1).fillna(0)
        
        angle = np.degrees(np.arctan2(df['prev_dy'], df['prev_dx']))
        df['direction'] = ((angle + 22.5) // 45).astype(int) % 8
        
        # Goal
        df['goal_distance'] = np.sqrt((105 - df['start_x'])**2 + (34 - df['start_y'])**2)
        df['goal_angle'] = np.degrees(np.arctan2(34 - df['start_y'], 105 - df['start_x']))
        
        # Time
        df['time_left'] = 5400 - df['time_seconds']
        
        # Episode
        df['pass_count'] = df.groupby('game_episode').cumcount() + 1
        
        return df
    
    def prepare_data(self, df):
        """마지막 패스 추출 & Feature/Target 분리"""
        train_last = df.groupby('game_episode').last().reset_index()
        
        feature_cols = [
            'start_x', 'start_y',
            'zone_x', 'zone_y',
            'direction',
            'goal_distance', 'goal_angle',
            'period_id', 'time_seconds', 'time_left',
            'pass_count',
            'prev_dx', 'prev_dy'
        ]
        
        X = train_last[feature_cols].values
        y = train_last[['end_x', 'end_y']].values
        groups = train_last['game_episode'].str.split('_').str[0].values
        
        print(f"\n  Episodes: {len(train_last):,}개")
        print(f"  Features: {len(feature_cols)}개")
        
        return X, y, groups, feature_cols

print("✅ FastExperiment 클래스 로드 완료!")

## 6. CatBoost 재현 (로컬 결과 검증)

In [ ]:
import catboost as cb

print("="*80)
print("CatBoost 재현 (Colab)")
print("="*80)
print("\n로컬 결과: CV 16.04 ± 0.27")
print("목표: 동일한 결과 재현")

# Setup
exp = FastExperiment(sample_frac=1.0, n_folds=5, random_state=42)

# Load data
train_df = exp.load_data(sample=False)
train_df = exp.create_features(train_df)
X, y, groups, feature_cols = exp.prepare_data(train_df)

# CatBoost
cat_params = {
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'random_state': 42,
    'verbose': 0,
    'task_type': 'GPU' if torch.cuda.is_available() else 'CPU'
}

gkf = GroupKFold(n_splits=5)
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups=groups)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model_x = cb.CatBoostRegressor(**cat_params)
    model_y = cb.CatBoostRegressor(**cat_params)
    
    model_x.fit(X_train, y_train[:, 0])
    model_y.fit(X_train, y_train[:, 1])
    
    pred_x = np.clip(model_x.predict(X_val), 0, 105)
    pred_y = np.clip(model_y.predict(X_val), 0, 68)
    
    dist = np.sqrt((pred_x - y_val[:, 0])**2 + (pred_y - y_val[:, 1])**2)
    cv = dist.mean()
    fold_scores.append(cv)
    
    print(f"  Fold {fold+1}: {cv:.4f}")

cv_mean = np.mean(fold_scores)
cv_std = np.std(fold_scores)

print(f"\n  Colab CV: {cv_mean:.4f} ± {cv_std:.4f}")
print(f"  로컬 CV:  16.0449 ± 0.2729")
print(f"  차이: {abs(cv_mean - 16.0449):.4f}")

if abs(cv_mean - 16.0449) < 0.01:
    print("\n✅ 재현 성공!")
else:
    print("\n⚠️ 차이 있음 (random seed 확인)")

## 7. Test 예측 & Submission

In [ ]:
print("="*80)
print("Test 예측")
print("="*80)

# Test data 로드
test_meta = pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission.csv')

print(f"  Test episodes: {len(test_meta):,}개")

# Test episodes 로드
test_episodes = []
for _, row in test_meta.iterrows():
    ep_df = pd.read_csv(row['path'])
    ep_df['game_episode'] = row['game_episode']
    test_episodes.append(ep_df)

test_df = pd.concat(test_episodes, ignore_index=True)
print(f"  Test 패스: {len(test_df):,}개")

# Features
test_df = exp.create_features(test_df)
test_last = test_df.groupby('game_episode').last().reset_index()
X_test = test_last[feature_cols].values

# Full train으로 재학습
print("\n  Full train 재학습...")
model_x_final = cb.CatBoostRegressor(**cat_params)
model_y_final = cb.CatBoostRegressor(**cat_params)

model_x_final.fit(X, y[:, 0])
model_y_final.fit(X, y[:, 1])

# Predict
pred_x = np.clip(model_x_final.predict(X_test), 0, 105)
pred_y = np.clip(model_y_final.predict(X_test), 0, 68)

# Submission
submission = pd.DataFrame({
    'game_episode': test_last['game_episode'],
    'end_x': pred_x,
    'end_y': pred_y
})

# Merge with sample_submission
submission = sample_sub[['game_episode']].merge(submission, on='game_episode', how='left')

# Save
submission.to_csv('submission_catboost_cv16.04.csv', index=False)
print("\n✅ submission_catboost_cv16.04.csv 저장 완료!")
print(f"  샘플 수: {len(submission)}")
print(f"  NaN: {submission.isna().sum().sum()}")

# Download
from google.colab import files
files.download('submission_catboost_cv16.04.csv')
print("\n📥 다운로드 시작!")

## 8. 다음 단계: Neural Network

Gap 확인 후 진행:
- Gap < 1.0 → TabNet, FT-Transformer 시도
- Gap > 2.0 → Regularization 강화, Feature engineering